# Imports

In [1]:
%cd ..

import sys
import os
from agents import *
from main import *

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_portfolio import StockPortfolioEnv

from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

# from pypfopt.efficient_frontier import EfficientFrontier
# from pypfopt import risk_models

import pyfolio
from pyfolio import timeseries

from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

%cd ./notebook

if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)



c:\Users\oscar\Github\Github\Research-project


C:\Users\oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


c:\Users\oscar\Github\Github\Research-project\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


c:\Users\oscar\Github\Github\Research-project\notebook


# Download and process data

In [3]:
df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2021-07-01',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

df.to_csv("df.csv.zip", compression="zip")

# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Prepare environment

In [5]:
train = data_split(df, '2009-01-01','2020-07-01')

stock_dimension = len(train.tic.unique())
state_space = stock_dimension

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = TIOnlyStockPortfolioEnv(df = train, **env_kwargs)

env_train, obs = e_train_gym.get_sb_env()

## Custom

In [2]:
e_train_gym = V1()

env_train, obs = e_train_gym.get_sb_env()

# Train agent

In [3]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 5000, "learning_rate": 0.001}

model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50)

{'batch_size': 128, 'buffer_size': 5000, 'learning_rate': 0.001}
Using cpu device
Logging to tensorboard_log/ddpg\ddpg_8


# Test Agent

In [ ]:
trade = data_split(df,'2020-07-01', '2021-07-01')
e_trade_gym = TIOnlyStockPortfolioEnv(df = trade, **env_kwargs)


df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg,
                        environment = e_trade_gym)



DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")


#baseline stats
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])
print("==============Get Baseline Stats===========")
stats = backtest_stats(baseline_df, value_col_name = 'close')

print("==============DRL Strategy Stats===========")
print(perf_stats_all)

## Custom

In [5]:
def DRL_prediction(model, environment):
        unique = environment.end_index - environment.start_index
        test_env, test_obs = environment.get_sb_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []
        test_env.reset()
        for i in range(unique):
            action, _states = model.predict(test_obs)
            if i == 10:
                print(action)
            #account_memory = test_env.env_method(method_name="save_asset_memory")
            #actions_memory = test_env.env_method(method_name="save_action_memory")
            test_obs, rewards, dones, info = test_env.step(action)
            if i == (unique - 2):
              account_memory = test_env.env_method(method_name="save_asset_memory")
              actions_memory = test_env.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        return account_memory[0], actions_memory[0]

def Market_returns(environment):
        unique = environment.end_index - environment.start_index
        test_env, test_obs = environment.get_sb_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []
        test_env.reset()
        for i in range(unique):
            action = [[1 for _ in range(30)]]
            #account_memory = test_env.env_method(method_name="save_asset_memory")
            #actions_memory = test_env.env_method(method_name="save_action_memory")
            test_obs, rewards, dones, info = test_env.step(action)
            if i == (unique - 2):
              account_memory = test_env.env_method(method_name="save_asset_memory")
        #       actions_memory = test_env.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        return account_memory[0]

# e_trade_gym = V1(training=False)

df_daily_return, df_actions = DRL_prediction(model=trained_ddpg, environment=V1(training=False))

DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func(returns=DRL_strat, factor_returns=DRL_strat, positions=None, transactions=None, turnover_denom="AGB")


market_df_daily_return = Market_returns(V1(training=False))

market_strat = convert_daily_return_to_pyfolio_ts(market_df_daily_return)
market_stats_all = perf_func(returns=market_strat, factor_returns=market_strat, positions=None, transactions=None, turnover_denom="AGB")

#baseline stats
# baseline_df = get_baseline(
#         ticker="^DJI", 
#         start = df_daily_return.loc[0,'date'],
#         end = df_daily_return.loc[len(df_daily_return)-1,'date'])
# print("==============Get Baseline Stats===========")
# stats = backtest_stats(baseline_df, value_col_name = 'close')

print("==============DRL Strategy Stats===========")
print(perf_stats_all)

print("==============Market Strategy Stats===========")
print(market_stats_all)

[[1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0.
  1. 1. 0. 1. 1. 0.]]
==============DRL Strategy Stats===========
Annual return           0.183683
Cumulative returns      0.402982
Annual volatility       0.281532
Sharpe ratio            0.740716
Calmar ratio            0.561161
Stability               0.760598
Max drawdown           -0.327327
Omega ratio             1.163959
Sortino ratio           1.030177
Skew                   -0.514836
Kurtosis               11.351824
Tail ratio              0.839781
Daily value at risk    -0.034642
Alpha                   0.000000
Beta                    1.000000
dtype: float64
==============Market Strategy Stats===========
Annual return           0.147234
Cumulative returns      0.317581
Annual volatility       0.287542
Sharpe ratio            0.622347
Calmar ratio            0.427654
Stability               0.662264
Max drawdown           -0.344283
Omega ratio             1.138861
Sortino ratio           0.863581
Skew  

In [5]:
def Market_returns(environment):
        unique = environment.end_index - environment.start_index
        test_env, test_obs = environment.get_sb_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []
        test_env.reset()
        for i in range(unique):
            action = [[1 for _ in range(30)]]
            #account_memory = test_env.env_method(method_name="save_asset_memory")
            #actions_memory = test_env.env_method(method_name="save_action_memory")
            test_obs, rewards, dones, info = test_env.step(action)
            print(action, rewards)
            if i == (unique - 2):
              account_memory = test_env.env_method(method_name="save_asset_memory")
        #       actions_memory = test_env.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        return account_memory[0]

market_df_daily_return = Market_returns(V1(training=False))

mark_func = timeseries.perf_stats 

market_strat = convert_daily_return_to_pyfolio_ts(market_df_daily_return)
market_stats_all = mark_func(returns=market_strat, factor_returns=market_strat, positions=None, transactions=None, turnover_denom="AGB")


print("==============Market Strategy Stats===========")
print(market_stats_all)
print(market_df_daily_return.loc[:,:])

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [982427.44]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [978138.1]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [971798.8]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [964021.94]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [930909.7]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [948189.6]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [926486.8]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [920116.56]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] [938730.25]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

# Plot test results

In [7]:
# %matplotlib inline

baseline_df = get_baseline(
        ticker='^DJI', start=df_daily_return.loc[0,'date'], end='2021-07-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

# with pyfolio.plotting.plotting_context(font_scale=1.1):
#         pyfolio.create_full_tear_sheet(returns = DRL_strat, benchmark_rets=baseline_returns, set_context=False)

# print(type(DRL_strat))
# # print(type(baseline_returns))
# DRL_strat.to_csv("results.csv")
# baseline_returns.to_csv("baseline.csv")



pyfolio.create_full_tear_sheet(returns=DRL_strat, benchmark_rets=baseline_returns, set_context=False)

# pyfolio.create_returns_tear_sheet(returns=DRL_strat, benchmark_rets=baseline_returns, set_context=False)

# pyfolio.plotting.plot_rolling_returns(DRL_strat, baseline_returns)


[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (681, 8)


Start date,2018-10-17
End date,2020-10-20
Total months,24
,Backtest
Annual return,15.255%
Cumulative returns,32.988%
Annual volatility,30.314%
Sharpe ratio,0.62
Calmar ratio,0.42
Stability,0.62
Max drawdown,-36.096%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,36.10,2020-02-19,2020-03-23,2020-08-12,126
1,16.96,2018-12-03,2018-12-24,2019-02-13,53
2,9.96,2020-09-02,2020-09-24,NaT,NaN
3,8.60,2018-10-17,2018-10-29,2018-12-03,34
4,7.30,2019-05-03,2019-06-03,2019-06-19,34


C:\Users\oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\plotting\_matplotlib\tools.py:395: MatplotlibDeprecationWarning: 
The is_last_row function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_last_row() instead.
  if ax.is_last_row():
C:\Users\oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\plotting\_matplotlib\tools.py:395: MatplotlibDeprecationWarning: 
The is_last_row function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_last_row() instead.
  if ax.is_last_row():
C:\Users\oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\plotting\_matplotlib\tools.py:395: MatplotlibDeprecationWarning: 
The is_last_row function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_last_row() instead.
  if ax.is_last_row():
C:\Users\oscar\AppData\Local\Programs\Py

Stress Events,mean,min,max
New Normal,0.07%,-13.71%,11.04%


C:\Users\oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


# Min Var plot (BROKEN)

In [ ]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.1))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()
    
    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value 
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)
    
portfolio=portfolio.T
portfolio.columns = ['account_value']

time_ind = pd.Series(df_daily_return.date)
td3_cumpod =(df_daily_return.daily_return+1).cumprod()-1
min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1
dji_cumpod =(baseline_returns+1).cumprod()-1

trace0_portfolio = go.Scatter(x = time_ind, y = td3_cumpod, mode = 'lines', name = 'TD3 (Portfolio Allocation)')
trace1_portfolio = go.Scatter(x = time_ind, y = dji_cumpod, mode = 'lines', name = 'DJIA')
trace2_portfolio = go.Scatter(x = time_ind, y = min_var_cumpod, mode = 'lines', name = 'Min-Variance')


fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)



fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=15,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2
        
    ),
)
#fig.update_layout(legend_orientation="h")
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
#fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
#    margin=dict(l=20, r=20, t=20, b=20),

    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Cumulative Return",
xaxis={'type': 'date', 
       'tick0': time_ind[0], 
        'tickmode': 'linear', 
       'dtick': 86400000.0 *80}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()